# Test ADJ Connection to ESPORTS JSON Repository

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import cx_Oracle
import os
import pandas as pd


os.environ['TNS_ADMIN'] = '/home/opc/adj_esportsdb'

In [ ]:

!pip install dataprep
!pip install dask
!pip install pandas_profiling
## install packages
!pip install -q scikit-learn
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

In [ ]:
import cx_Oracle
import yaml
import os
from pathlib import Path
home = str(Path.home())

def process_yaml():
	with open("../config.yaml") as file:
		return yaml.safe_load(file)


class OracleJSONDatabaseConnection:
    def __init__(self, data=process_yaml()):
        # wallet location (default is HOME/wallets/wallet_X)
        os.environ['TNS_ADMIN'] = '{}/{}'.format(home, process_yaml()['WALLET_DIR'])
        print(os.environ['TNS_ADMIN'])
        self.pool = cx_Oracle.SessionPool(data['db']['username'], data['db']['password'], data['db']['dsn'],
            min=1, max=4, increment=1, threaded=True,
            getmode=cx_Oracle.SPOOL_ATTRVAL_WAIT
        )
        print('Connection successful.')



    def close_pool(self):
        self.pool.close()
        print('Connection pool closed.')



    def insert(self, collection_name, json_object_to_insert):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)

        try:
            x_collection.insertOne(json_object_to_insert)
            print('[DBG] INSERT {} OK'.format(json_object_to_insert))
        except cx_Oracle.IntegrityError:
            print('[DBG] INSERT {} ERR'.format(json_object_to_insert))
            return 0
        self.pool.release(connection)
        return 1



    def delete(self, collection_name, on_column, on_value):
        connection = self.pool.acquire()
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)
        qbe = {on_column: on_value}
        x_collection.find().filter(qbe).remove()
        self.pool.release(connection)



    def get_connection(self):
        return self.pool.acquire() 



    def close_connection(self, conn_object):
        self.pool.release(conn_object)



    def get_collection_names(self):
        connection = self.pool.acquire()
        returning_object = connection.getSodaDatabase().getCollectionNames(startName=None, limit=0)
        self.pool.release(connection)
        return returning_object



    def open_collection(self, collection_name):
        connection = self.pool.acquire()
        returning_object = self.pool.acquire().getSodaDatabase().openCollection(collection_name)
        self.pool.release(connection)
        return returning_object



def test_class():
    object = OracleJSONDatabaseConnection()
    print(object.pool)
    object.close_pool()

In [ ]:
print(os.environ['TNS_ADMIN'])

In [ ]:
db = OracleJSONDatabaseConnection()
print(db.get_collection_names())

In [ ]:
data = db.open_collection('predictor_liveclient')
all_data = list()
for doc in data.find().getCursor():
    content = doc.getContent()
    all_data.append(content)

print('Data length: {}'.format(len(all_data)))

In [ ]:
df = pd.read_json(json.dumps(all_data), orient='records')

df.head(5)


In [ ]:
df.describe()

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
report = ProfileReport(df)
#report #uncomment to display all.

In [ ]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [ ]:
df = TabularDataset(df)

# drop columns we don't want
df = df.drop(columns=['identifier', 'participantId'])

train = df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

train.head(5)

In [ ]:
label = 'winner'

In [ ]:
save_path = './autogluon_regressor_trained_models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train)

In [ ]:
y_test = test[label]  # values to predict
test_data_nolabel = test.drop(columns=[label])  # delete label column to prove we're not cheating, also drop identifier column
test_data_nolabel.head()

In [ ]:
predictor = TabularPredictor.load(save_path)

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


In [ ]:
predictor.leaderboard(test, silent=False)

In [ ]:
predictor.feature_importance(test)